In [10]:
# 1. Cài đặt các gói hỗ trợ
!apt-get update
!apt-get install -y wget unzip

# 2. Tải và cài đặt Google Chrome Stable (Bản chính thức)
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get install -y ./google-chrome-stable_current_amd64.deb

# 3. Cài đặt thư viện Python cần thiết
!pip install selenium webdriver-manager

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (3,763 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3

In [15]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

# --- CẤU HÌNH ---
TARGET_URL = "https://spiderum.com/bai-dang/Lam-the-nao-de-toi-uu-hoa-viec-phat-trien-ban-than-lva"

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1920,1080') # Màn hình lớn để dễ tắt popup
    options.add_argument('--lang=vi')

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def close_popup_if_exists(driver):
    """Hàm chuyên trị tắt Pop-up quảng cáo"""
    print("-> Đang kiểm tra Pop-up quảng cáo...")
    time.sleep(5) # Chờ popup hiện ra hẳn

    try:
        # Cách 1: Gửi phím ESC (Cách hiệu quả nhất với mọi loại popup)
        ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        print("-> Đã nhấn ESC để tắt Popup.")
        time.sleep(1)

        # Cách 2: Tìm nút đóng (X) cụ thể (Dự phòng nếu ESC không ăn thua)
        # Tìm nút có icon đóng hoặc class thường thấy của modal close
        close_buttons = driver.find_elements(By.CSS_SELECTOR, "div[class*='close'], button[class*='close'], svg[data-icon='times']")
        if len(close_buttons) > 0:
            for btn in close_buttons:
                if btn.is_displayed():
                    btn.click()
                    print("-> Đã click nút đóng (X).")
                    break
    except Exception as e:
        print(f"-> Không tắt được Popup (hoặc không có): {e}")

def scrape_spiderum_final():
    driver = setup_driver()
    data = []

    try:
        print(f"Đang truy cập: {TARGET_URL}")
        driver.get(TARGET_URL)

        # 1. Tắt quảng cáo UFO
        close_popup_if_exists(driver)

        # 2. Cuộn trang để load comment
        print("Đang cuộn trang...")
        last_height = driver.execute_script("return document.body.scrollHeight")
        for i in range(10): # Cuộn 10 lần
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

            # Thử tắt popup lần nữa (đề phòng nó hiện lại khi cuộn)
            if i == 1:
                ActionChains(driver).send_keys(Keys.ESCAPE).perform()

            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # 3. Quét dữ liệu (Dùng Selector cực rộng để bắt dính)
        print("Đang quét dữ liệu...")

        # Tìm tất cả các khối có vẻ là comment
        # Mẹo: Spiderum comment thường nằm trong các thẻ div có class chứa chữ 'comment'
        comment_blocks = driver.find_elements(By.XPATH, "//div[contains(@class, 'comment')] | //div[contains(@class, 'content-body')]")

        print(f"-> Tìm thấy {len(comment_blocks)} khối dữ liệu tiềm năng.")

        unique_comments = set() # Dùng set để lọc trùng lặp

        for block in comment_blocks:
            try:
                text = block.text.strip()
                # Lọc rác: Comment phải dài hơn 10 ký tự và không phải nút bấm
                if len(text) > 10 and "Đăng nhập" not in text and "Trả lời" != text:
                    # Tách tên và nội dung (Giả sử dòng đầu là tên, các dòng sau là nội dung)
                    parts = text.split('\n', 1)
                    if len(parts) >= 2:
                        author = parts[0].strip()
                        content = parts[1].strip()

                        # Tạo khóa để kiểm tra trùng
                        key = f"{author}_{content[:20]}"
                        if key not in unique_comments:
                            data.append([author, content])
                            unique_comments.add(key)
            except:
                continue

    except Exception as e:
        print(f"Lỗi: {e}")
    finally:
        driver.quit()

    return data

# --- CHẠY ---
if __name__ == "__main__":
    result = scrape_spiderum_final()

    if result:
        print(f"\n✅ LẤY ĐƯỢC {len(result)} COMMENT!")
        # In thử 5 cái đầu tiên
        for idx, r in enumerate(result[:5]):
            print(f"{idx+1}. {r[0]}: {r[1][:50]}...")

        with open("spiderum_final.csv", "w", encoding="utf-8-sig", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["Author", "Content"])
            writer.writerows(result)
        print("\n-> File 'spiderum_final.csv' đã sẵn sàng tải về.")
    else:
        print("\n❌ Vẫn chưa lấy được. Spiderum đang chặn khá gắt.")

Đang truy cập: https://spiderum.com/bai-dang/Lam-the-nao-de-toi-uu-hoa-viec-phat-trien-ban-than-lva
-> Đang kiểm tra Pop-up quảng cáo...
-> Đã nhấn ESC để tắt Popup.
Đang cuộn trang...
Đang quét dữ liệu...
-> Tìm thấy 61 khối dữ liệu tiềm năng.

✅ LẤY ĐƯỢC 23 COMMENT!
1. Gửi: Hot nhất
Mới nhất
mazechan
3 tháng 3 2020
Upvote k...
2. Hot nhất: Mới nhất...
3. mazechan: 3 tháng 3 2020
Upvote kèm theo chúc em nhiều sức k...
4. Scarlet: Đã chỉnh sửa 28 tháng 9 2021
ôi anh dậy sớm thế, a...
5. mazechan: 3 tháng 3 2020
anh chứ ko phải chị em ui  và anh đ...

-> File 'spiderum_final.csv' đã sẵn sàng tải về.
